In [31]:
!pip install arxiv

In [32]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [33]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [34]:
wiki.name

'wikipedia'

In [35]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size =1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,OllamaEmbeddings())
retriver = vectordb.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002432B78DA10>, search_kwargs={})

In [36]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(retriver,"langsmith_search","Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")
retriver_tool.name

'langsmith_search'

In [37]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [38]:
tools=[wiki,arxiv,retriver_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\suhas\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000242E7AC1620>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vecto

In [39]:
from langchain_community.chat_models import ChatOllama

In [40]:
import os
llm=ChatOllama(model="llama2")
llm

ChatOllama()

In [41]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [42]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [43]:
from langchain.agents import AgentExecutor
agent_execute = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_execute

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [ ]:
agent_execute.invoke({"input":"Tell me about Langsmith ?"})



> Entering new AgentExecutor chain...


In [45]:
agent_execute.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Ah, an excellent question! Paper 1605.08386 is actually a research article titled "A Neural Algorithm of Object Recognition" by Yann LeCun et al., published in the IEEE Transactions on Neural Networks in 1990.

The paper introduces the LeNet-1 neural network architecture, which is considered one of the earliest and most influential deep learning models for image recognition tasks. The authors propose a new approach to object recognition based on a multilayer perceptron (MLP) trained on a large dataset of images. They demonstrate that their algorithm can recognize handwritten digits with high accuracy, achieving state-of-the-art performance at the time.

The key innovation of LeNet-1 is its use of multiple layers of nonlinear transformations to extract features from the input data. By combining convolutional and pooling layers, the network learns to extract complex features such as edges, shapes, and textures, which are then used for classificati

{'input': "What's the paper 1605.08386 about?",
 'output': '\nAh, an excellent question! Paper 1605.08386 is actually a research article titled "A Neural Algorithm of Object Recognition" by Yann LeCun et al., published in the IEEE Transactions on Neural Networks in 1990.\n\nThe paper introduces the LeNet-1 neural network architecture, which is considered one of the earliest and most influential deep learning models for image recognition tasks. The authors propose a new approach to object recognition based on a multilayer perceptron (MLP) trained on a large dataset of images. They demonstrate that their algorithm can recognize handwritten digits with high accuracy, achieving state-of-the-art performance at the time.\n\nThe key innovation of LeNet-1 is its use of multiple layers of nonlinear transformations to extract features from the input data. By combining convolutional and pooling layers, the network learns to extract complex features such as edges, shapes, and textures, which are t